## **Importer les bibliothèques nécessaires:**

In [ ]:
import numpy as np
import pandas as pd

## **Définition de la classe Station_Simulation:**

In [ ]:
class Station_Simulation:
    def __init__(self): 
        self.horloge=0.0                    #horloge de la simulation 
        self.num_arrivee=0                  #nombre total des arrivées
        self.t_arrivee=self.gen_int_arr()   #temps de l'arrivée prochaine
        self.t_depart1=float('inf')         #temps de depart de la cellule 1
        self.t_depart2=float('inf')         #temps de depart de la cellule 2
        self.dep_sum1=0                     #Sum of service times by teller 1
        self.dep_sum2=0                     #Sum of service times by teller 2
        self.etat_cel1=0                    #L'état de la cellule 1
        self.etat_cel2=0                    #L'état de la cellule 1
        self.t_temps_attente=0.0            #temps d'attente totale
        self.num_in_q=0                     #Nombre des voitures dans le parking
        self.n_p=0                          #Nombre des voitures qui ont utilisé le parking
        self.num_departs1=0                 #nombre des départs à partir de le cellule 1 
        self.num_departs2=0                 #nombre des départs à partir de le cellule 2 
        self.clients_perdus=0               #voitures quittant sans service
        self.num_in_system=0                #nombre des voitures dans le système

    def TNOW(self):                                                       
        t_next_event=min(self.t_arrivee,self.t_depart1,self.t_depart2)  
        self.t_temps_attente += (self.num_in_q*(t_next_event-self.horloge))
        self.horloge=t_next_event
                
        if self.t_arrivee<self.t_depart1 and self.t_arrivee<self.t_depart2:
            self.arrivee()
        elif self.t_depart1<self.t_arrivee and self.t_depart1<self.t_depart2:
            self.Cellule1()
        else:
            self.Cellule2()

    def arrivee(self):              
        self.num_arrivee += 1
        self.num_in_system += 1

        if self.num_in_q == 0:         #planifier l'évenement prochain
            if self.etat_cel1==1 and self.etat_cel2==1:   #Si les deux cellules sont ocuppées
                self.num_in_q+=1
                self.n_p+=1
                self.t_arrivee=self.horloge+self.gen_int_arr()
                
                
            elif self.etat_cel1==0 and self.etat_cel2==0:   #Si les deux cellules sont libres
                
                if np.random.choice([0,1])==1: 
                    self.etat_cel1=1
                    self.dep1= self.gen_depart_cellule1()
                    self.dep_sum1 += self.dep1
                    self.t_depart1=self.horloge + self.dep1
                    self.t_arrivee=self.horloge+self.gen_int_arr()

                else:
                    self.etat_cel2=1
                    self.dep2= self.gen_depart_cellule2()
                    self.dep_sum2 += self.dep2
                    self.t_depart2=self.horloge + self.dep2
                    self.t_arrivee=self.horloge+self.gen_int_arr()

                    
            elif self.etat_cel1==0 and self.etat_cel2 ==1:       #Si la cellule1 est libre
                self.dep1= self.gen_depart_cellule1()
                self.dep_sum1 += self.dep1
                self.t_depart1=self.horloge + self.dep1
                self.t_arrivee=self.horloge+self.gen_int_arr()
                self.etat_cel1=1
            else:                                              
                self.dep2= self.gen_depart_cellule2()
                self.dep_sum2 += self.dep2
                self.t_depart2=self.horloge + self.dep2
                self.t_arrivee=self.horloge+self.gen_int_arr()
                self.etat_cel2=1
        
        elif self.num_in_q < 10 and self.num_in_q >= 1:       #Si le parking n'est pas totalement plein
            self.num_in_q+=1
            self.n_p+=1                             
            self.t_arrivee=self.horloge + self.gen_int_arr()
            
        else:
          self.clients_perdus+=1

    def Cellule1(self):                #depart de la cellule 1
        self.num_departs1 += 1
        if self.num_in_q>0:
            self.dep1= self.gen_depart_cellule1()
            self.dep_sum1 += self.dep1
            self.t_depart1=self.horloge + self.dep1
            self.num_in_q-=1
        else:
            self.t_depart1=float('inf') 
            self.etat_cel1=0                  
    
    def Cellule2(self):                #depart de la cellule 2
        self.num_departs2 += 1
        if self.num_in_q>0:
            self.dep2= self.gen_depart_cellule2()
            self.dep_sum2 += self.dep2
            self.t_depart2=self.horloge + self.dep2
            self.num_in_q-=1
        else:
            self.t_depart2=float('inf')
            self.etat_cel2=0

    def gen_int_arr(self):                                             #générer les arrivées
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 10.71)
    
    def gen_depart_cellule1(self):                                #génerer les départs de la cellule1
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 10)
    
    def gen_depart_cellule2(self):                                #génerer les départs de la cellule2
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 10)

## **Simulation et Résultats**

In [ ]:
s=Station_Simulation()
df=pd.DataFrame(columns=['Temps des inter-arrivées moyen','Temps de service de la cellule 1 moyen','Temps de service de la cellule 2 moyen',
                         'taux d’occupation de la cellule 1',"taux d’occupation de la cellule 2",'Nombre des utilisations du parking',
                         "Temps d'attente moyen",'Clients Perdus'])


for i in range(50):
    np.random.seed(i)
    s.__init__()
    while s.horloge <= 720 :
        s.TNOW() 
    a=pd.Series([s.horloge/s.num_arrivee,s.dep_sum1/s.num_departs1,s.dep_sum2/s.num_departs2,
                 s.dep_sum1/s.horloge,s.dep_sum2/s.horloge,s.n_p,s.t_temps_attente/s.n_p,s.clients_perdus],index=df.columns)
    df=df.append(a,ignore_index=True) 

In [ ]:
df

,Temps des inter-arrivées moyen,Temps de service de la cellule 1 moyen,Temps de service de la cellule 2 moyen,taux d’occupation de la cellule 1,taux d’occupation de la cellule 2,Nombre des utilisations du parking,Temps d'attente moyen,Clients Perdus
0,11.250873,8.581239,9.029220,0.381359,0.401268,11.0,4.197945,0.0
1,9.738393,12.441960,9.386863,0.552484,0.534054,29.0,8.190551,0.0
2,9.550625,10.764072,8.706382,0.541571,0.461721,22.0,8.255148,0.0
3,10.006766,11.285688,11.409126,0.533422,0.585478,37.0,11.143558,0.0
4,9.872835,11.422466,9.840828,0.523009,0.491552,25.0,9.145601,0.0
5,10.055213,13.825087,8.921437,0.572882,0.505237,33.0,12.960157,0.0
6,11.341596,9.469448,7.623876,0.378328,0.357109,11.0,7.500635,0.0
7,9.874136,11.628753,9.529071,0.572934,0.495568,28.0,9.585053,0.0
8,9.307668,8.661587,8.738382,0.471183,0.463476,14.0,3.822320,0.0
9,11.846233,9.501883,8.513347,0.407625,0.329874,9.0,3.266228,0.0


In [ ]:
df.mean(axis=0)

Temps des inter-arrivées moyen            10.907380
Temps de service de la cellule 1 moyen    10.534716
Temps de service de la cellule 2 moyen     9.559431
taux d’occupation de la cellule 1          0.456799
taux d’occupation de la cellule 2          0.446731
Nombre des utilisations du parking        18.940000
Temps d'attente moyen                      7.166825
Clients Perdus                             0.000000
dtype: float64